In [126]:
!pip install oracledb

In [127]:
import oracledb
import json
import random
import hashlib
import getpass
from datetime import datetime
import time
import os

In [128]:
# ------------------- Configuração visual -------------------

In [129]:
def limpar_tela():
    try: os.system('cls' if os.name == 'nt' else 'clear')
    except: print("\n" * 50)

In [130]:
def pausar():
    input("\nPressione [ENTER] para continuar...")

In [131]:
# ------------------- Conexão com o banco -------------------

In [132]:
def get_conn():
    try:
        with open("secret.txt", "r", encoding="utf-8") as f:
            creds = json.load(f)
        return oracledb.connect(user=creds["user"], password=creds["password"], dsn=creds["dsn"])
    except Exception as e:
        print(f"❌ Erro de Conexão: {e}")
        return None

In [133]:
# ------------------- Funções Auxiliares -------------------

In [134]:
def hash_password(password):
    return hashlib.sha256(password.encode('utf-8')).hexdigest()

In [135]:
def verify_password(plain_password, stored_hash):
    new_hash = hash_password(plain_password)
    return new_hash == stored_hash or plain_password == stored_hash

In [136]:
def get_next_id(cur, table_name, id_column):
    try:
        cur.execute(f"SELECT MAX({id_column}) FROM {table_name}")
        row = cur.fetchone()
        return int(row[0]) + 1 if row and row[0] else 1
    except: return 1

In [137]:
def get_user_level_info(cur, user_id):
    try:
        cur.execute("""
            SELECT n.ID_NIVEL, n.NOME_NIVEL FROM NIVEL_USUARIO nu
            JOIN NIVEL n ON nu.ID_NIVEL = n.ID_NIVEL
            WHERE nu.ID_USUARIO = :1 ORDER BY nu.DATA_HORA_NIVEL DESC
        """, (user_id,))
        row = cur.fetchone()
        return row if row else (1, "Iniciante")
    except: return (1, "Iniciante")

In [138]:
# ------------------- Login e Cadastro -------------------

In [139]:
def register_user(is_admin_creating=False):
    conn = get_conn(); 
    if not conn: return
    cur = conn.cursor()
    
    limpar_tela()
    if is_admin_creating: print("\n--- ADMIN: CRIAR NOVO USUÁRIO ---")
    else: print("\n=== CADASTRO LEVELUP ===")
        
    nome = input("Nome: ").strip()

    while True:
        email = input("Email: ").strip()
        if "@" not in email: print("❌ Email inválido."); continue
        cur.execute("SELECT ID_USUARIO FROM USUARIO WHERE EMAIL_USUARIO = :1", (email,))
        if cur.fetchone(): print("❌ Email já existe."); continue
        break
    
    if is_admin_creating: senha = input("Senha temporária: ").strip()
    else: senha = getpass.getpass("Senha: ").strip()
    
    try:
        new_id = get_next_id(cur, "USUARIO", "ID_USUARIO")
        cur.execute("""
            INSERT INTO USUARIO (ID_USUARIO, NOME_USUARIO, EMAIL_USUARIO, SENHA_USUARIO, PLANO_USUARIO)
            VALUES (:1, :2, :3, :4, 'BRONZE')
        """, (new_id, nome, email, hash_password(senha)))
        cur.execute("INSERT INTO NIVEL_USUARIO VALUES (:1, 1, SYSDATE)", (new_id,))
        conn.commit()
        print(f"✅ Usuário criado com sucesso! ID: {new_id}")
    except Exception as e: print(f"Erro: {e}")
    finally: 
        cur.close(); conn.close()
        if not is_admin_creating: pausar()

In [140]:
def login_user():
    conn = get_conn(); 
    if not conn: return None
    cur = conn.cursor()
    
    limpar_tela()
    print("\n=== LOGIN ===")
    email = input("Email: ").strip()
    if not email: cur.close(); conn.close(); return None
    senha = getpass.getpass("Senha: ").strip()

    cur.execute("SELECT ID_USUARIO, NOME_USUARIO, SENHA_USUARIO, PLANO_USUARIO FROM USUARIO WHERE EMAIL_USUARIO = :1", (email,))
    row = cur.fetchone()

    if row and verify_password(senha, row[2]):
        role = 'admin' if email == 'admin@levelup.com' else 'user'
        has_ai = row[3] in ['PRATA', 'OURO']
        has_spec = row[3] == 'OURO'
        
        lvl_id, lvl_nome = get_user_level_info(cur, row[0])
        
        user = {
            "id": row[0], "name": row[1], "email": email, "plan": row[3], 
            "level_id": lvl_id, "level_name": lvl_nome, 
            "role": role, "has_ai": has_ai, "has_spec": has_spec
        }
        print(f"✅ Bem-vindo, {row[1]}!")
        time.sleep(1)
        cur.close(); conn.close(); return user
    
    print("❌ Acesso negado.")
    pausar()
    cur.close(); conn.close(); return None

In [141]:
# ------------------- CHAT + INDICAÇÃO DE CHALLENGE -------------------

In [142]:
def play_challenge(user):
    conn = get_conn(); cur = conn.cursor()
    limpar_tela()
    print("\n🎮 --- ÁREA DE DESAFIOS ---")
    
    try:
        cur.execute("SELECT ID_CHALLENGE, NOME_CHALLENGE, DESCRICAO_CHALLENGE FROM CHALLENGE ORDER BY ID_CHALLENGE")
        challs = cur.fetchall()
        if not challs: print("⚠️ Nenhum desafio disponível."); pausar(); cur.close(); conn.close(); return

        for c in challs:
            print(f"[{c[0]}] {c[1]}")
            
        cid = input("\nDigite o ID do desafio para jogar (0 sair): ")
        if cid == '0': cur.close(); conn.close(); return
        
        sel = next((c for c in challs if str(c[0]) == cid), None)
        
        if sel:
            limpar_tela()
            print(f"\n🔥 DESAFIO: {sel[1]}")
            print(f"📝 Tarefa: {sel[2]}")
            print("\n(Simulação: Digite a resposta correta para ganhar XP)")
            
            resp = input("Sua Resposta: ").strip()
            
            if resp: 
                print("\n✅ Resposta Enviada! Analisando...")
                time.sleep(1.5)
                print("🌟 PARABÉNS! Você completou o desafio.")
                
                current_lvl = user['level_id']
                next_lvl = current_lvl + 1
                
                if next_lvl <= 15: 
                    try:
                        cur.execute("INSERT INTO NIVEL_USUARIO VALUES (:1, :2, SYSDATE)", (user['id'], next_lvl))
                        conn.commit()
                        
                        cur.execute("SELECT NOME_NIVEL FROM NIVEL WHERE ID_NIVEL = :1", (next_lvl,))
                        new_lvl_name = cur.fetchone()[0]
                        
                        user['level_id'] = next_lvl
                        user['level_name'] = new_lvl_name
                        print(f"\n🚀 LEVEL UP! Você subiu para: {new_lvl_name}")
                        
                    except Exception as e: print(f"Erro ao subir nível: {e}")
                else:
                    print("\n🏆 Você já está no nível MÁXIMO (Lendário)!")
            else:
                print("❌ Resposta em branco. Tente novamente.")
        else:
            print("❌ ID inválido.")
            
    except Exception as e: print(f"Erro: {e}")
    finally:
        cur.close(); conn.close(); pausar()

In [143]:
def save_chat_db(user_id, conteudo, rec_id=None):
    conn = get_conn(); cur = conn.cursor()
    try:
        chat_id = f"C{int(datetime.now().timestamp())}"
        cur.execute("INSERT INTO CONVERSA VALUES (:1, :2, SYSDATE, :3)", (chat_id, user_id, conteudo))
        if rec_id:
            nid = get_next_id(cur, "CONVERSA_INDICA_CHALLENGE", "ID_CONVERSA_INDICA_CHALLENGE")
            cur.execute("INSERT INTO CONVERSA_INDICA_CHALLENGE VALUES (:1, :2, :3)", (nid, chat_id, rec_id))
        conn.commit()
    except Exception as e: print(f"Erro log: {e}")
    finally: cur.close(); conn.close()

In [144]:
def ia_chat_simulation(user):
    limpar_tela()
    print("\n🤖 CHAT IA - 'sair' para encerrar.")
    
    conn = get_conn(); cur = conn.cursor()
    
    cur.execute("SELECT ID_CURSO, NOME_CURSO FROM CURSO")
    cursos = cur.fetchall()
    rec_curso = random.choice(cursos) if cursos else None
    
    rec_desafio = None
    if rec_curso:
        cur.execute("SELECT ID_CHALLENGE, NOME_CHALLENGE FROM CHALLENGE WHERE ID_CURSO = :1", (rec_curso[0],))
        challs = cur.fetchall()
        rec_desafio = random.choice(challs) if challs else None
        
    cur.close(); conn.close()

    history = []
    respostas = ["Muito interessante.", "Entendi, continue.", "Soft skills são essenciais.", "Ótimo ponto!"]
    print("IA: Olá! Como posso ajudar nos seus estudos?")

    while True:
        msg = input("Você: ").strip()
        if msg.lower() == 'sair': break
        resp = f"IA: {random.choice(respostas)}"
        print(resp); history.append(f"U:{msg}|{resp}")
    
    full = "||".join(history)
    
    if rec_curso and rec_desafio:
        print(f"\n🤖 IA: Com base na conversa, recomendo o curso: '{rec_curso[1]}'")
        print(f"       Para praticar, tente o desafio: '{rec_desafio[1]}'")
        save_chat_db(user['id'], f"[IA - Rec: {rec_curso[1]}] {full}", rec_desafio[0])
    else:
        save_chat_db(user['id'], f"[IA] {full}")
        
    pausar()

In [145]:
def specialist_chat_simulation(user):
    limpar_tela()
    conn = get_conn(); cur = conn.cursor()
    print("\n🎓 ESCOLHA UM CURSO")
    try:
        cur.execute("SELECT c.ID_CURSO, c.NOME_CURSO, p.NOME_PROFESSOR FROM CURSO c JOIN PROFESSOR p ON c.ID_PROFESSOR = p.ID_PROFESSOR")
        cursos = cur.fetchall()
        for c in cursos: print(f"[{c[0]}] {c[1]} ({c[2]})")
    except: print("Erro busca."); cur.close(); conn.close(); return
    cid = input("\nID Curso (0 sair): ")
    if cid not in [str(c[0]) for c in cursos]: cur.close(); conn.close(); return
    cur.execute("SELECT ID_CHALLENGE, NOME_CHALLENGE FROM CHALLENGE WHERE ID_CURSO = :1", (cid,))
    recs = cur.fetchall(); rec = random.choice(recs) if recs else None
    prof = next(c[2] for c in cursos if str(c[0]) == cid)
    limpar_tela()
    print(f"\n💬 Falando com {prof}..."); history=[]
    while True:
        msg = input("Você: ").strip()
        if msg.lower() == 'sair': break
        print(f"{prof}: Entendido. Na prática é assim..."); history.append(f"U:{msg}|P:OK")
    if rec: print(f"\n🎓 {prof}: Faça o desafio '{rec[1]}'")
    save_chat_db(user['id'], f"[MENTORIA {prof}] {'||'.join(history)}", rec[0] if rec else None)
    cur.close(); conn.close(); pausar()

In [146]:
def upgrade_plan(user):
    limpar_tela()
    conn = get_conn(); cur = conn.cursor()
    print(f"\n=== UPGRADE (Atual: {user['plan']}) ===")
    print("1- BRONZE (Grátis) | 2- PRATA (R$19.90) | 3- OURO (R$39.90)")
    op = input("Escolha: ")
    novo = "BRONZE" if op=="1" else "PRATA" if op=="2" else "OURO" if op=="3" else None

    if not novo or novo == user['plan']:
        print("⚠️ Cancelado."); cur.close(); conn.close(); pausar(); return

    if novo == 'BRONZE':
        print("Alterando..."); time.sleep(1); print("✅ Feito.")
    else:
        valor = 19.90 if novo == 'PRATA' else 39.90
        print(f"Total: R$ {valor:.2f}")
        while True:
            cartao = input("💳 Cartão (só números): ").replace(" ","").replace("-","")
            if cartao.isdigit() and len(cartao) >= 13: break
            print("Inválido.")
        while True:
            cvv = input("🔒 CVV (3 dig): ").strip()
            if cvv.isdigit() and len(cvv) == 3: break
            print("Inválido (3 dígitos).")
        print("Processando...", end="", flush=True); time.sleep(1.5); print(" APROVADO!")

    try:
        cur.execute("UPDATE USUARIO SET PLANO_USUARIO = :1 WHERE ID_USUARIO = :2", (novo, user['id']))
        conn.commit()
        user['plan'] = novo; user['has_ai'] = novo in ['PRATA','OURO']; user['has_spec'] = novo == 'OURO'
        print(f"🎉 Parabéns! Plano {novo} ativo.")
    except Exception as e: print(f"Erro: {e}")
    finally: cur.close(); conn.close(); pausar()

In [147]:
# ------------------- ADMIN -------------------

In [148]:
def admin_users_menu():
    conn = get_conn(); cur = conn.cursor()
    while True:
        limpar_tela()
        print("\n--- GERENCIAR USUÁRIOS ---")
        print("1. Listar | 2. Criar | 3. Editar | 4. Deletar | 0. Voltar")
        op = input("Opção: ")
        if op == "0": break
        elif op == "1":
            cur.execute("SELECT ID_USUARIO, NOME_USUARIO, PLANO_USUARIO FROM USUARIO ORDER BY ID_USUARIO")
            print("\nLISTA:")
            for u in cur.fetchall(): print(f"[{u[0]}] {u[1]} ({u[2]})")
            pausar()
        elif op == "2": register_user(True)
        elif op == "3": 
            uid = input("ID: ")
            cur.execute("SELECT NOME_USUARIO FROM USUARIO WHERE ID_USUARIO=:1", (uid,))
            if cur.fetchone():
                plano = input("Novo Plano (BRONZE/PRATA/OURO): ").upper()
                cur.execute("UPDATE USUARIO SET PLANO_USUARIO=:1 WHERE ID_USUARIO=:2", (plano, uid))
                conn.commit(); print("✅ Sucesso.")
            else: print("❌ Não achou.")
            pausar()
        elif op == "4":
            uid = input("ID para deletar: ")
            if input("⚠️ Tem certeza? (s/n): ") == 's':
                try:
                    cur.execute("SELECT ID_CONVERSA FROM CONVERSA WHERE ID_USUARIO = :1", (uid,))
                    chats = cur.fetchall()
                    for chat in chats:
                        cur.execute("DELETE FROM CONVERSA_INDICA_CHALLENGE WHERE ID_CONVERSA = :1", (chat[0],))
                    cur.execute("DELETE FROM CONVERSA WHERE ID_USUARIO=:1", (uid,))
                    cur.execute("DELETE FROM NIVEL_USUARIO WHERE ID_USUARIO=:1", (uid,))
                    cur.execute("DELETE FROM CHALLENGE WHERE ID_USUARIO=:1", (uid,))
                    cur.execute("DELETE FROM USUARIO WHERE ID_USUARIO=:1", (uid,))
                    
                    conn.commit()
                    print("✅ Usuário deletado com sucesso.")
                except Exception as e: 
                    print(f"❌ Erro ao deletar: {e}")
            pausar()
    cur.close(); conn.close()

In [149]:
def admin_professors_menu():
    conn = get_conn(); cur = conn.cursor()
    while True:
        limpar_tela()
        print("\n--- GERENCIAR PROFESSORES ---")
        print("1. Listar | 2. Adicionar | 3. Deletar | 0. Voltar")
        op = input("Opção: ")
        if op == "0": break
        elif op == "1":
            cur.execute("SELECT ID_PROFESSOR, NOME_PROFESSOR FROM PROFESSOR")
            for p in cur.fetchall(): print(f"[{p[0]}] {p[1]}")
            pausar()
        elif op == "2":
            n = input("Nome: "); e = input("Email: ")
            nid = get_next_id(cur, "PROFESSOR", "ID_PROFESSOR")
            cur.execute("INSERT INTO PROFESSOR VALUES (:1, :2, 'Geral', :3, '123')", (nid, n, e))
            conn.commit(); print("✅ Feito."); pausar()
        elif op == "3":
            pid = input("ID: ")
            try:
                cur.execute("DELETE FROM PROFESSOR WHERE ID_PROFESSOR=:1", (pid,))
                conn.commit(); print("✅ Deletado.")
            except: print("❌ Erro (Vínculos?).")
            pausar()
    cur.close(); conn.close()

In [150]:
def admin_content_menu():
    conn = get_conn(); cur = conn.cursor()
    while True:
        limpar_tela()
        print("\n--- CONTEÚDO ---")
        print("1. Listar Cursos | 2. Criar Curso | 3. Criar Desafio | 0. Voltar")
        op = input("Opção: ")
        if op == "0": break
        elif op == "1":
            cur.execute("SELECT ID_CURSO, NOME_CURSO FROM CURSO")
            for c in cur.fetchall(): print(f"[{c[0]}] {c[1]}")
            pausar()
        elif op == "2":
            cur.execute("SELECT ID_PROFESSOR, NOME_PROFESSOR FROM PROFESSOR")
            for p in cur.fetchall(): print(f"[{p[0]}] {p[1]}")
            pid = input("ID Prof: "); nome = input("Curso: ")
            nid = get_next_id(cur, "CURSO", "ID_CURSO")
            cur.execute("INSERT INTO CURSO VALUES (:1, :2, :3, 'Desc')", (nid, pid, nome))
            conn.commit(); print("✅ Criado."); pausar()
        elif op == "3":
            cur.execute("SELECT ID_CURSO, NOME_CURSO FROM CURSO")
            for c in cur.fetchall(): print(f"[{c[0]}] {c[1]}")
            cid = input("ID Curso: "); nome = input("Desafio: ")
            nid = get_next_id(cur, "CHALLENGE", "ID_CHALLENGE")
            cur.execute("INSERT INTO CHALLENGE VALUES (:1, 99, :2, :3, 'Desc')", (nid, cid, nome))
            conn.commit(); print("✅ Criado."); pausar()
    cur.close(); conn.close()

In [151]:
def admin_export_menu():
    conn = get_conn(); cur = conn.cursor()
    while True:
        limpar_tela()
        print("\n--- EXPORTS JSON ---")
        print("1. Users | 2. Desafios | 3. Logs | 0. Voltar")
        op = input("Opção: ")
        if op == "0": break
        fname = ""
        if op == "1": cur.execute("SELECT * FROM USUARIO"); fname="export_users.json"
        elif op == "2": cur.execute("SELECT * FROM CHALLENGE"); fname="export_challenges.json"
        elif op == "3": 
            cur.execute("SELECT c.DATA_HORA_CONVERSA, u.NOME_USUARIO, c.CONTEUDO_CONVERSA FROM CONVERSA c JOIN USUARIO u ON c.ID_USUARIO = u.ID_USUARIO")
            fname="export_logs.json"
        else: continue
        data = [dict(zip([d[0] for d in cur.description], row)) for row in cur.fetchall()]
        with open(fname, "w", encoding='utf-8') as f: json.dump(data, f, default=str, indent=2)
        print(f"✅ {fname} gerado!"); pausar()
    cur.close(); conn.close()

In [152]:
# ------------------- Menus principais -------------------

In [153]:
def admin_menu(user):
    while True:
        limpar_tela()
        print(f"\n=== PAINEL ADMIN ({user['name']}) ===")
        print("1. Usuários")
        print("2. Professores")
        print("3. Conteúdo")
        print("4. Exports")
        print("0. Logout")
        op = input("Opção: ")
        if op == "1": admin_users_menu()
        elif op == "2": admin_professors_menu()
        elif op == "3": admin_content_menu()
        elif op == "4": admin_export_menu()
        elif op == "0": break

In [154]:
def user_menu(user):
    conn = get_conn(); cur = conn.cursor()
    
    while True:
        limpar_tela()
        print(f"\n=== 🚀 {user['name']} ({user['plan']}) | Nível: {user['level_name']} ===")
        opcoes = []
        opcoes.append(("Ver Lista de Desafios", "view_challenges"))
        opcoes.append(("Upgrade de Plano", "upgrade"))
        if user['has_ai']:
            opcoes.append(("Chat IA (Tira-Dúvidas)", "chat_ia"))
            
        if user['has_spec']:
            opcoes.append(("Chat com Especialista (Mentoria)", "chat_spec"))

        opcoes.append(("🎮 JOGAR DESAFIO (Ganhar Nível)", "play_game"))

        for i, (titulo, acao) in enumerate(opcoes, start=1):
            print(f"{i} - {titulo}")        
        print("0 - Sair")
        escolha_txt = input("\nOpção: ").strip()
        if escolha_txt == "0":
            break     
        if not escolha_txt.isdigit():
            print("❌ Opção inválida."); time.sleep(1); continue  
        escolha_num = int(escolha_txt)
        if 1 <= escolha_num <= len(opcoes):
            acao_selecionada = opcoes[escolha_num - 1][1]
            if acao_selecionada == "view_challenges":
                print("\n--- DESAFIOS ---")
                cur.execute("SELECT NOME_CHALLENGE FROM CHALLENGE")
                for r in cur.fetchall(): print(f"- {r[0]}")
                pausar()
                
            elif acao_selecionada == "upgrade":
                upgrade_plan(user)
                
            elif acao_selecionada == "chat_ia":
                ia_chat_simulation(user)
                
            elif acao_selecionada == "chat_spec":
                specialist_chat_simulation(user)
                
            elif acao_selecionada == "play_game":
                play_challenge(user)
                
        else:
            print("❌ Opção inválida."); time.sleep(1)
            
    cur.close(); conn.close()

In [155]:
def main():
    while True:
        limpar_tela()
        print("\n=== LEVELUP ===")
        print("1- Login")
        print("2- Cadastro")
        print("0- Sair")
        op = input("Opção: ")
        if op == "1":
            u = login_user()
            if u: (admin_menu(u) if u['role']=='admin' else user_menu(u))
        elif op == "2": register_user()
        elif op == "0": break

In [156]:
if __name__ == "__main__":
    main()


=== LEVELUP ===
1- Login
2- Cadastro
0- Sair

=== LOGIN ===
✅ Bem-vindo, Administrador!

=== PAINEL ADMIN (Administrador) ===
1. Usuários
2. Professores
3. Conteúdo
4. Exports
0. Logout

--- GERENCIAR USUÁRIOS ---
1. Listar | 2. Criar | 3. Editar | 4. Deletar | 0. Voltar

LISTA:
[1] Ana Silva (BRONZE)
[2] Bruno Costa (PRATA)
[3] Carla Souza (OURO)
[4] Daniel Rocha (OURO)
[5] Elaine Melo (PRATA)
[6] Fernando Lima (BRONZE)
[7] Gabriela Pires (OURO)
[8] Henrique Dias (BRONZE)
[9] Isabela Ramos (PRATA)
[10] João Pedro (OURO)
[11] Karen Silva (BRONZE)
[12] Lucas Almeida (PRATA)
[13] Mariana Souza (OURO)
[14] Nicolas Antunes (PRATA)
[15] Olivia Martins (BRONZE)
[99] Administrador (OURO)
[100] Lucas (OURO)
[102] clei (PRATA)

--- GERENCIAR USUÁRIOS ---
1. Listar | 2. Criar | 3. Editar | 4. Deletar | 0. Voltar

--- GERENCIAR USUÁRIOS ---
1. Listar | 2. Criar | 3. Editar | 4. Deletar | 0. Voltar
✅ Usuário deletado com sucesso.

--- GERENCIAR USUÁRIOS ---
1. Listar | 2. Criar | 3. Editar | 4. D